- 사용할 data에 대해 먼저 설명을 해보겠습니다. 해당 data는 UCSD의 Jianmo Ni가 배포한 것([link](http://deepyeti.ucsd.edu/jianmo/amazon/))으로 Amazon의 책 판매 부문과 관련된 data입니다.
- 데이터는 크게 두 부분으로 구성되어 있습니다. 1) user가 각 item들에 대해 5-stars scale로 rating을 한 log, 2) 각 item들(Amazon에서 판매하는 책)에 대한 meta-data로 이루어져 있습니다.
- 원활한 설명을 위해 순서를 조금 바꾸어 설명하겠습니다. 두 번째로 언급했던 item의 meta data를 먼저 처리하고, 이를 바탕으로 첫 번째로 언급했던 rating log를 처리하겠습니다.
- Preprocessing의 결과로 1) item들의 meta data -> item들의 ranking value, 2) rating log -> dictionary로 가공됩니다. 전처리 결과 각 user와 item들은 (0, 73058), (0, 160808) range의 숫자를 부여받습니다. rating log의 preprocessing의 결과인 dictionary는 training data와 test data로 나뉘게 되며 (key: user number, values: item numbers or 반대)의 형태로 가공됩니다. 

In [1]:
import os
import json
import gzip
import re
import math
import pandas as pd
import numpy as np
from collections import defaultdict

In [2]:
# Data Load
data = []
with gzip.open('./data/meta_Books.json.gz') as f:
    for l in f:
        data.append(json.loads(l.strip()))

In [3]:
data[0].keys()

dict_keys(['category', 'tech1', 'description', 'fit', 'title', 'also_buy', 'tech2', 'brand', 'feature', 'rank', 'also_view', 'main_cat', 'similar_item', 'date', 'price', 'asin', 'imageURL', 'imageURLHighRes'])

- 해당 data를 전처리하여 사용하려는 목적은 다음과 같습니다. user와 item으로 Bipartite하게 나뉘어진 graph data에 대해, 각 user와 item이 connect 되었는지를 rating의 유무로 판단하여 관계를 포착하는 GCN(Graph Convolutional Network) based model을 사용할 때, 성능을 올리기 위한 feature로 무엇을 사용할 수 있을지를 찾아보는 것이 목적입니다.
- item의 meta-data는 위에 ```keys()```를 통해 나온 결과처럼 다양한 data들이 포함되어 있습니다. 하지만 그 중 ranking을 사용하려는 이유는 가장 feature가 될 가능성이 있어보여서였습니다.
- Category와 title, description, brand(작가의 이름) 등을 사용하여 user가 특정 부류의 책을 좋아하는 지를 포착해볼까 하는 생각도 했지만, 해당 feature들은 data에 결측치가 너무 많아 일반적으로 적용하기는 힘들어보였습니다.
- 다음으로는 also_view, also_buy라는 feature가 있었지만, 이를 사용하는 것은 큰 의미가 없다고 생각했습니다. 우리가 보통 물건을 생각하는 과정을 고려해보았을 때, 여러가지 물건들을 '보고', 그 중에서 더 맘에 드는 것들을 골라 '구매'하고, 그 중에서 더 강렬한 감정(실망 혹은 기쁨)을 느낀 물건들에 후기를 달고 rating을 합니다. 이런 일련의 과정들을 생각해보았을 때, (보고, 구매하고, 평가를 한 것)들 중에서 가장 user의 선호를 잘 나타내는 것은 '평가를 한 물건들'이라고 생각하였습니다. 그런데 아래에서 rating log data를 전처리한 후 학습에 사용할 예정이기 때문에, also_view와 also_buy를 사용하는 것은 성능에 유의미한 향상을 가져오지는 못할 것이라 판단했습니다.

In [ ]:
# Ranking Extract
ranking_dict = dict() # key: asin(item code), value: rank
except_rank = dict() # key: asin(item code), value: defective value
for i in data:
    try:
        rank = re.sub(r'[^0-9]', '', i['rank'])
        if rank == '':
            except_rank[i['asin']] = i['rank']
        else:
            ranking_dict[i['asin']] = rank
    except TypeError:
        except_rank[i['asin']] = i['rank']

- rank 역시 결측치가 어느정도 있어서 이를 처리해주어야 합니다. Regular expression을 사용하여 숫자만 추출하여 ```ranking_dict```에 저장하고, 숫자가 없거나 비어있는 경우엔 ```except_rank```에 저장해주었습니다.

In [4]:
### Data Load
print('Data Load')
rating_data = pd.read_csv('./data/Books.csv', names = ['item', 'user', 'rating']) # five-stars rating
rating_data.head()

Data Load


,item,user,rating
0001713353,A1C6M8LCIX4M6M,5.0,1123804800
0001713353,A1REUF3A1YCPHM,5.0,1112140800
0001713353,A1YRBRK2XM5D5,5.0,1081036800
0001713353,A1V8ZR5P78P4ZU,5.0,1077321600
0001713353,A2ZB06582NXCIV,5.0,1475452800


- 다음 data는 rating log data입니다. raw data는 ```head()```로 출력한 바와 같습니다. 전처리 후에는 rating을 했는지 안 했는지의 여부만 dictionary의 형태로 남게 되고 몇점으로 rating 했는지에 대한 정보는 소실됩니다. 해당 정보(별점)까지 사용하기 위해서는 graph의 edge에 별점만큼 weight을 주어야 하는데, 그렇게 되면 모델이 훨씬 복잡해지기에 단순한 GCN based model을 사용한다는 목적에 부합하게 rating 여부만 사용하였습니다.(edge_weight을 사용하는 graph neural network는 GAT라는 model이 있습니다만 학습에 걸리는 시간이 현재 사용 예정인 model의 몇 백배 수준입니다.)

In [ ]:
### user and item Grouping
print('Data Grouping')
rating_item_dict = dict(list(rating_data['user'].groupby(rating_data['item']))) # key: item code, value: user code
rating_item_set = set()
for i in rating_item_dict:
    if (len(rating_item_dict[i]) >= 50) & (i in ranking_dict):
        rating_item_set.add(i) # collect items that were rated by users more than 50 times and have ranking value

rating_user_dict = dict(list(rating_data['item'].groupby(rating_data['user']))) # key: user code, value: item code
rating_user_set = set()
for i in rating_user_dict:
    if len(rating_user_dict[i]) >= 50: # collect users that rated items more than 50 times
        rating_user_set.add(i)

- rating log data는 일단 user와 item의 고유 code를 사용하여 각각 ```groupby()```함수를 통해 dictionary data로 grouping을 해줍니다. 이때 key는 user(or item)의 code이고, value는 item(or user)의 code입니다.
- 그 다음 set에서는 50회 이상 rating을 한 user와, 50회 이상 rated된 item만 저장합니다. item의 경우 특별히 이에 더해 ranking dictionary에 해당 data가 있는지 확인(ranking의 결측치 확인)하는 과정도 추가됩니다.

In [ ]:
### Construct user and item set for filtering
processed_user = set()
processed_item = set()

for i in rating_item_set: # item set rated by users more than 50 times and have ranking value.
    user_from_item = set(rating_item_dict[i].values)
    temp = (user_from_item & rating_user_set)
    if len(temp) >= 2:
        processed_item.add(i)

for i in rating_user_set: # user set rated more than 50
    item_from_user = set(rating_user_dict[i].values)
    temp = (item_from_user & processed_item)
    if len(temp) >= 2:
        processed_user.add(i)

- 위의 과정에선 Bipartite graph로 만들어지지 않는 data를 filtering합니다. 풀어서 설명하면, 이전 과정에서 50회 이상 rating을 한 user와 50회 이상 rated된 item만을 filtering하였는데, 해당 data들을 key로 삼아 기존 dictionary에서 조회한다면 당연히 value에는 50회 이상 rating을 하지 않은 user나 item이 조회될 것입니다. 이를 아래에서 training과 test data로 나눌 때 참조하여 filtering하기 위해 ```processed_user```와 ```processed_item``` set을 구성합니다.

In [ ]:
processed_user_list = list(processed_user) # User codes are the elements, we will use this list's index as user's number
processed_item_list = list(processed_item) # Item codes are the elements, we will use this list's index as item's number

# training set
train_user_item_dict = defaultdict(set) # key : user, value : item
train_item_user_dict = defaultdict(set) # key : item, value : user
# test set
test_user_item_dict = defaultdict(set)
test_item_user_dict = defaultdict(set)
# overall set
rating_set_all = defaultdict(set)

- ```processed_user_list```와 ```processed_item_list```를 만듭니다. 이는 아직까지 고유 code로 저장되어있는 각 user와 item에, 별도의 숫자로 mapping하는 과정없이, 해당 list의 index를 mapping되는 숫자로 사용하기 위함입니다.
- 그 외에 training과 test set으로 저장하기 위한 dictionary를 선언하는데, user가 key이고 item이 value인 ```user_item_dict```와, item이 key이고 user가 value인 ```item_user_dict```를 나누어 선언합니다.

In [ ]:
### Change user and item codes into number and split into training and test dict ###
print('Matrix making start')
temp = set()
for i in processed_user: # filtered users
    item_from_user = set(rating_user_dict[i].values)
    temp = (item_from_user & processed_item) # filtered items
    count = len(temp)
    if count >= 2:
        user_index = int(processed_user_list.index(i))
        item_codes = list(temp)
        
        ######################## 1st round ########################
        not_included = [] # It means 'not included' in 'train_item_user_dict' yet -> should be assigned to that dict.
        already_included = [] # It means 'already included' in 'train_item_user_dict'
        for j in item_codes:
            item_index = int(processed_item_list.index(j))
            rating_set_all[user_index].add(item_index)
            
            if (item_index not in train_item_user_dict): # Is item_index in train_item_user_dict as 'key'?
                not_included.append(item_index)# if not, there is no any item in train_item_user_dict related with\
                                               # this item key, fill train_item_user_dict with it first.
            else:
                already_included.append(item_index)
        for j in not_included:
            train_user_item_dict[user_index].add(j)
            train_item_user_dict[j].add(user_index)
            
        ######################## 2nd round ########################
        test_cut = math.ceil(count*0.2) # I want to split 8 : 2 = train : test dictionary
        if len(already_included) != 0: 
            length = len(already_included) - test_cut # length for elements will go to training dict.
            if length > 0:
                for j in already_included[:length]:
                    train_user_item_dict[user_index].add(j)
                    train_item_user_dict[j].add(user_index)
                for j in already_included[length:]:
                    test_user_item_dict[user_index].add(j)
                    test_item_user_dict[j].add(user_index)
            else: # length == 0 -> assign to test dict
                for j in already_included:
                    test_user_item_dict[user_index].add(j)
                    test_item_user_dict[j].add(user_index)

- 먼저 user를 ```processed_user```에서 받아옵니다. 그리고 해당 user-key의 value로 딸려있는 item들을 가져와서 ```processed_item```을 이용하여 한번 filtering을 진행합니다.
- 1st round에서는 ```training_item_user_dict```(item이 key, user가 value)의 key인 item의 수가 ```processed_item```의 length만큼 되도록 하기 위해, test dataset보다 training dataset에 우선적으로 배정할 user-item pair를 선별합니다. 이렇게 하는 이유는 해당 data를 사용할 GCN based model인 LR-GCCF(Linear Residual Graph Convolutional Collaborative Filtering) model에서 ```training_user_item_dict```와 ```training_item_user_dict```을 사용하여 \[user $\times$ item\], \[item $\times$ user\] shape의 matrix를 만들고, 이를 각각 \[item $\times$ n\], \[user $\times$ n\]의 embedding matrix와 matrix multiplication을 하는 과정이 있기 때문입니다. 이때 matrix의 column과 row부분의 dimension이 맞아야 matrix multiplication을 할 수 있기 때문에 위와 같은 과정을 진행합니다.
- 2nd round에서는 남은 data들을 가지고 training data와 test data에 8:2의 비율로 적절히 나눠줍니다.
- 위와 같은 처리를 해줄 경우, test를 할 때 test data에 user에 대응되는 item이 없는 경우도 존재하지만, precision이 아니라 recall을 metric으로 사용할 것이기에 성능평가를 할 때 문제가 생기지 않습니다.

In [ ]:
print('Count start')
train_count = 0
test_count = 0
for i in train_user_item_dict: # count train data values
    train_count = train_count + len(train_user_item_dict[i])
for i in test_user_item_dict: # count test data values
    test_count = test_count + len(test_user_item_dict[i])

- training data와 test data에 할당된 value(item)의 수를 카운트하고 저장하는 과정입니다.

In [ ]:
print('Make ranking list')
ranking_list = []
for i in processed_item_list: # i is a item code
    ranking_list.append(ranking_dict[i]) # append ranking value in order(processed_item_list is ordered)

- 이 과정에선 ranking을 item의 저장순서에 맞춰 list로 다시 저장합니다. ```processed_item_list```의 index를 각 item의 mapping된 number로 사용하기에, 이를 기준으로 ```ranking_list```를 만들어줍니다.

In [ ]:
print('Save as npy file')
np.save('./training_set.npy', [train_user_item_dict, train_item_user_dict, train_count])
np.save('./testing_set.npy', [test_user_item_dict, test_item_user_dict, test_count])
np.save('./rating_set_all.npy', rating_set_all)
np.save('./ranking_list.npy', ranking_list)

- 전처리 후엔 model에서 data를 load하여 사용할 수 있도록 ```.npy``` file format으로 저장합니다.